In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("test").getOrCreate()

In [3]:
data_list = [
    (1, "Adam Smith", 900000, "ABC"),
    (2, "Brian Jones", 800000, "XYZ"),
    (3, "Chris Milne", 580000, "ABC"),
    (4, "David Harris", 80000, "PQE"),
]
df = spark.createDataFrame(data_list, ["id", "name", "salary", "company"])
df.show()

+---+------------+------+-------+
| id|        name|salary|company|
+---+------------+------+-------+
|  1|  Adam Smith|900000|    ABC|
|  2| Brian Jones|800000|    XYZ|
|  3| Chris Milne|580000|    ABC|
|  4|David Harris| 80000|    PQE|
+---+------------+------+-------+



In [4]:
new_df = df.withColumn("increment", expr("salary * 10 / 100")).withColumn(
    "new_salary", expr("salary + increment")
)
new_df.show()
new_df.explain(extended=True)

+---+------------+------+-------+---------+----------+
| id|        name|salary|company|increment|new_salary|
+---+------------+------+-------+---------+----------+
|  1|  Adam Smith|900000|    ABC|  90000.0|  990000.0|
|  2| Brian Jones|800000|    XYZ|  80000.0|  880000.0|
|  3| Chris Milne|580000|    ABC|  58000.0|  638000.0|
|  4|David Harris| 80000|    PQE|   8000.0|   88000.0|
+---+------------+------+-------+---------+----------+

== Parsed Logical Plan ==
'Project [id#0L, name#1, salary#2L, company#3, increment#25, ('salary + 'increment) AS new_salary#31]
+- Project [id#0L, name#1, salary#2L, company#3, (cast((salary#2L * cast(10 as bigint)) as double) / cast(100 as double)) AS increment#25]
   +- LogicalRDD [id#0L, name#1, salary#2L, company#3], false

== Analyzed Logical Plan ==
id: bigint, name: string, salary: bigint, company: string, increment: double, new_salary: double
Project [id#0L, name#1, salary#2L, company#3, increment#25, (cast(salary#2L as double) + increment#25) A

In [5]:
df.select(
    "id",
    "name",
    "salary",
    expr("salary * 10 / 100").alias("increment"),
    expr("salary + increment").alias("new_salary"),
).show()

+---+------------+------+---------+----------+
| id|        name|salary|increment|new_salary|
+---+------------+------+---------+----------+
|  1|  Adam Smith|900000|  90000.0|  990000.0|
|  2| Brian Jones|800000|  80000.0|  880000.0|
|  3| Chris Milne|580000|  58000.0|  638000.0|
|  4|David Harris| 80000|   8000.0|   88000.0|
+---+------------+------+---------+----------+



## Performance Impact

In [6]:
df = spark.createDataFrame([Row(id=1, name="abc"), Row(id=2, name="xyz")])
dummy_col_list = ["foo1", "foo2", "foo3", "foo4", "foo5"]
for col_name in dummy_col_list:
    df = df.withColumn(col_name, lit(None).cast("string"))
df.show()

+---+----+----+----+----+----+----+
| id|name|foo1|foo2|foo3|foo4|foo5|
+---+----+----+----+----+----+----+
|  1| abc|null|null|null|null|null|
|  2| xyz|null|null|null|null|null|
+---+----+----+----+----+----+----+



In [7]:
df.explain("extended")

== Parsed Logical Plan ==
Project [id#91L, name#92, foo1#95, foo2#99, foo3#104, foo4#110, cast(null as string) AS foo5#117]
+- Project [id#91L, name#92, foo1#95, foo2#99, foo3#104, cast(null as string) AS foo4#110]
   +- Project [id#91L, name#92, foo1#95, foo2#99, cast(null as string) AS foo3#104]
      +- Project [id#91L, name#92, foo1#95, cast(null as string) AS foo2#99]
         +- Project [id#91L, name#92, cast(null as string) AS foo1#95]
            +- LogicalRDD [id#91L, name#92], false

== Analyzed Logical Plan ==
id: bigint, name: string, foo1: string, foo2: string, foo3: string, foo4: string, foo5: string
Project [id#91L, name#92, foo1#95, foo2#99, foo3#104, foo4#110, cast(null as string) AS foo5#117]
+- Project [id#91L, name#92, foo1#95, foo2#99, foo3#104, cast(null as string) AS foo4#110]
   +- Project [id#91L, name#92, foo1#95, foo2#99, cast(null as string) AS foo3#104]
      +- Project [id#91L, name#92, foo1#95, cast(null as string) AS foo2#99]
         +- Project [id#91L,

Okay. There are multiple Project nodes, so what? It's not in the Physical Plan, and Spark will finally execute the selected Physical Plan. So, there shouldn't be any performance regression because of this during execution.

Well, the performance degradation happens before it even reaches the Physical Plan.

### Cause of Performance Degradation
Each time withColumn is used to add a column in the dataframe, Spark’s Catalyst optimizer re-evaluates the whole plan repeatedly. This adds up fast and strains performance.

**The surprising part? You might not notice it until you dig into Spark’s Logical Plans.**

This issue is not so obvious because it doesn't show up in the SparkUI. Your job that might take only 5mins to complete, can end up taking 5x more time because of multiple withColumn.

### Solution #1: Using .withColumns() for Spark >= 3.3
Starting from Spark 3.3, withColumns() transformation is available to use, that takes a dictionary of string and Column datatype.

Only a single Project node present in the Logical Plan.


In [8]:
df = spark.createDataFrame([Row(id=1, name="abc"), Row(id=2, name="xyz")])
dummy_col_val_map = {
    "foo1": lit(None).cast("string"),
    "foo2": lit(None).cast("string"),
    "foo3": lit(None).cast("string"),
    "foo4": lit(None).cast("string"),
    "foo5": lit(None).cast("string"),
}

# Adding columns using withColumns
df1 = df.withColumns(dummy_col_val_map)

# Checking both Analytical and Physical Plan
df1.explain("extended")

== Parsed Logical Plan ==
Project [id#154L, name#155, cast(null as string) AS foo1#158, cast(null as string) AS foo2#159, cast(null as string) AS foo3#160, cast(null as string) AS foo4#161, cast(null as string) AS foo5#162]
+- LogicalRDD [id#154L, name#155], false

== Analyzed Logical Plan ==
id: bigint, name: string, foo1: string, foo2: string, foo3: string, foo4: string, foo5: string
Project [id#154L, name#155, cast(null as string) AS foo1#158, cast(null as string) AS foo2#159, cast(null as string) AS foo3#160, cast(null as string) AS foo4#161, cast(null as string) AS foo5#162]
+- LogicalRDD [id#154L, name#155], false

== Optimized Logical Plan ==
Project [id#154L, name#155, null AS foo1#158, null AS foo2#159, null AS foo3#160, null AS foo4#161, null AS foo5#162]
+- LogicalRDD [id#154L, name#155], false

== Physical Plan ==
*(1) Project [id#154L, name#155, null AS foo1#158, null AS foo2#159, null AS foo3#160, null AS foo4#161, null AS foo5#162]
+- *(1) Scan ExistingRDD[id#154L,name#1

### Solution #2: Using .select() with an alias
Another way to achieve the same is via `.select` with `alias` or `.selectExpr()`

In [9]:
df2 = df.select(
    "*", *[cvalue.alias(cname) for cname, cvalue in dummy_col_val_map.items()]
)

df2.explain("extended")

== Parsed Logical Plan ==
'Project [*, cast(null as string) AS foo1#170, cast(null as string) AS foo2#171, cast(null as string) AS foo3#172, cast(null as string) AS foo4#173, cast(null as string) AS foo5#174]
+- LogicalRDD [id#154L, name#155], false

== Analyzed Logical Plan ==
id: bigint, name: string, foo1: string, foo2: string, foo3: string, foo4: string, foo5: string
Project [id#154L, name#155, cast(null as string) AS foo1#170, cast(null as string) AS foo2#171, cast(null as string) AS foo3#172, cast(null as string) AS foo4#173, cast(null as string) AS foo5#174]
+- LogicalRDD [id#154L, name#155], false

== Optimized Logical Plan ==
Project [id#154L, name#155, null AS foo1#170, null AS foo2#171, null AS foo3#172, null AS foo4#173, null AS foo5#174]
+- LogicalRDD [id#154L, name#155], false

== Physical Plan ==
*(1) Project [id#154L, name#155, null AS foo1#170, null AS foo2#171, null AS foo3#172, null AS foo4#173, null AS foo5#174]
+- *(1) Scan ExistingRDD[id#154L,name#155]



### FAQs
Every time I explain this, there are some follow up questions that engineers ask:

**Is this the only case when `.withColumn` is used in for loop?**

No. The same issue happens when we use multiple `.withColumn` outside loop also. We can look into the Logical Plan again to verify it.

In [10]:
df4 = (
    df.withColumn("foo1", lit(None).cast("string"))
    .withColumn("foo2", lit(None).cast("string"))
    .withColumn("foo3", lit(None).cast("string"))
    .withColumn("foo4", lit(None).cast("string"))
    .withColumn("foo5", lit(None).cast("string"))
)

df4.explain("extended")

== Parsed Logical Plan ==
Project [id#154L, name#155, foo1#182, foo2#186, foo3#191, foo4#197, cast(null as string) AS foo5#204]
+- Project [id#154L, name#155, foo1#182, foo2#186, foo3#191, cast(null as string) AS foo4#197]
   +- Project [id#154L, name#155, foo1#182, foo2#186, cast(null as string) AS foo3#191]
      +- Project [id#154L, name#155, foo1#182, cast(null as string) AS foo2#186]
         +- Project [id#154L, name#155, cast(null as string) AS foo1#182]
            +- LogicalRDD [id#154L, name#155], false

== Analyzed Logical Plan ==
id: bigint, name: string, foo1: string, foo2: string, foo3: string, foo4: string, foo5: string
Project [id#154L, name#155, foo1#182, foo2#186, foo3#191, foo4#197, cast(null as string) AS foo5#204]
+- Project [id#154L, name#155, foo1#182, foo2#186, foo3#191, cast(null as string) AS foo4#197]
   +- Project [id#154L, name#155, foo1#182, foo2#186, cast(null as string) AS foo3#191]
      +- Project [id#154L, name#155, foo1#182, cast(null as string) AS f

**Should we not use .withColumn at all then?**

If the number of columns being added are fairly low, we can use it, it wouldn't make much of a difference.
But if you are planning to write a code, that you think can further be extended based on the upcoming requirements, I would recommend using .withColumns or other 2 options.

**How many .withColumn are too many that can cause degradation?**

There are no specific numbers of columns, but if you have like 100s of withColumn with some transformation logics, chances are your Spark Job can do so much better.

**How can we look into SparkUI then if this is the issue?**

The issue won't be so easily visible on SparkUI, the starting point is to compare the Job Uptime and the time taken by the Jobs in Spark UI Jobs tab.

If all the jobs are finishing quickly but total Uptime is significantly higher, chances are multiple withColumn are the potential cause.